In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
import numpy as np
%matplotlib inline

In [2]:
df1 = pd.read_csv('./data_in/SRE.csv')

In [3]:
df1.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-09-04,116.370003,117.449997,116.099998,116.989998,113.424011,2142900
1,2018-09-05,117.010002,118.279999,116.690002,118.139999,114.538956,1856300
2,2018-09-06,118.279999,118.970001,117.699997,118.029999,114.432312,1203800
3,2018-09-07,117.139999,117.589996,116.379997,116.849998,113.288277,1216500
4,2018-09-10,117.309998,118.199997,116.910004,117.599998,114.015419,1072000


In [4]:
df1.shape

(250, 7)

In [5]:
df1.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
245,2019-08-26,139.070007,141.919998,138.990005,141.789993,141.789993,1282700
246,2019-08-27,142.500000,142.860001,141.029999,141.429993,141.429993,1633400
247,2019-08-28,141.570007,142.149994,141.009995,142.000000,142.000000,1216400
248,2019-08-29,142.330002,142.660004,141.639999,142.240005,142.240005,1444300
249,2019-08-30,142.910004,142.910004,141.320007,141.630005,141.630005,1040300


In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 7 columns):
Date         250 non-null object
Open         250 non-null float64
High         250 non-null float64
Low          250 non-null float64
Close        250 non-null float64
Adj Close    250 non-null float64
Volume       250 non-null int64
dtypes: float64(5), int64(1), object(1)
memory usage: 13.8+ KB


In [7]:
df1['Date'] = pd.to_datetime(df1['Date'])

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 7 columns):
Date         250 non-null datetime64[ns]
Open         250 non-null float64
High         250 non-null float64
Low          250 non-null float64
Close        250 non-null float64
Adj Close    250 non-null float64
Volume       250 non-null int64
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 13.8 KB


In [9]:
df1.columns = df1.columns.str.upper()
df1.columns = df1.columns.str.replace(' ', '_')

In [10]:
df1.head()

,DATE,OPEN,HIGH,LOW,CLOSE,ADJ_CLOSE,VOLUME
0,2018-09-04,116.370003,117.449997,116.099998,116.989998,113.424011,2142900
1,2018-09-05,117.010002,118.279999,116.690002,118.139999,114.538956,1856300
2,2018-09-06,118.279999,118.970001,117.699997,118.029999,114.432312,1203800
3,2018-09-07,117.139999,117.589996,116.379997,116.849998,113.288277,1216500
4,2018-09-10,117.309998,118.199997,116.910004,117.599998,114.015419,1072000


In [11]:
df = df1.loc[:,['DATE', 'ADJ_CLOSE', 'VOLUME']]
df['HL_PCT'] = ((df1['HIGH'] - df1['LOW']) / df1['CLOSE']) * 100
df['DELTA_PCT'] = ((df1['CLOSE'] - df1['OPEN']) / df1['OPEN']) *100

In [12]:
df.shape

(250, 5)

In [13]:
df.head()

,DATE,ADJ_CLOSE,VOLUME,HL_PCT,DELTA_PCT
0,2018-09-04,113.424011,2142900,1.153944,0.532779
1,2018-09-05,114.538956,1856300,1.345858,0.965727
2,2018-09-06,114.432312,1203800,1.076001,-0.211363
3,2018-09-07,113.288277,1216500,1.035515,-0.247568
4,2018-09-10,114.015419,1072000,1.096933,0.247208


In [14]:
df['DATE'] = pd.to_datetime(df1['DATE'])

In [15]:
df.head()

,DATE,ADJ_CLOSE,VOLUME,HL_PCT,DELTA_PCT
0,2018-09-04,113.424011,2142900,1.153944,0.532779
1,2018-09-05,114.538956,1856300,1.345858,0.965727
2,2018-09-06,114.432312,1203800,1.076001,-0.211363
3,2018-09-07,113.288277,1216500,1.035515,-0.247568
4,2018-09-10,114.015419,1072000,1.096933,0.247208


In [16]:
df.info()

#No missing values.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 5 columns):
DATE         250 non-null datetime64[ns]
ADJ_CLOSE    250 non-null float64
VOLUME       250 non-null int64
HL_PCT       250 non-null float64
DELTA_PCT    250 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 9.9 KB


In [17]:
df = df.set_index(['DATE'])

In [18]:
df.head()

,ADJ_CLOSE,VOLUME,HL_PCT,DELTA_PCT
DATE,,,,
2018-09-04,113.424011,2142900,1.153944,0.532779
2018-09-05,114.538956,1856300,1.345858,0.965727
2018-09-06,114.432312,1203800,1.076001,-0.211363
2018-09-07,113.288277,1216500,1.035515,-0.247568
2018-09-10,114.015419,1072000,1.096933,0.247208


In [19]:
y = df['ADJ_CLOSE']

In [20]:
X = df[['VOLUME', 'HL_PCT', 'DELTA_PCT']]

### Developing the testing/training sets based on the pre-determined size (`train_test_split`).

In [21]:
#Set size of training set at 20% and round up.
train_size = math.ceil(0.20 * len(X))
print('Size of Training Set: {}'.format(train_size))

#Take the most recent samples as the testing set.
y_test = y[-train_size:]
X_test = X[-train_size:]

#Take the oldest samples as the training set (i.e., all of the samples, minus the testing samples.)
y_train = y[:(df.shape[0]-train_size)]
X_train = X[:(df.shape[0]-train_size)]

Size of Training Set: 50


### Model 1:  Simple Linear Model
`https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html`

In [22]:
lr = LinearRegression().fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print('The R^2 of the linear model is: {0:.3f}'.format(lr.score(X_test, y_test)))

The R^2 of the linear model is: -70.462


In [23]:
df_y_pred_lr = pd.DataFrame(y_pred_lr, index=X_test.index, columns=['PRED_ADJ_CLOSE'])
df_y_pred_lr.head(10)

,PRED_ADJ_CLOSE
DATE,
2019-06-21,117.785939
2019-06-24,122.564533
2019-06-25,121.058025
2019-06-26,116.726072
2019-06-27,113.264711
2019-06-28,119.069993
2019-07-01,118.650340
2019-07-02,122.681508
2019-07-03,122.289192


### Model 2:  K-Nearest Neighbors (kNN) Regression 
`https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html`

In [24]:
knn = KNeighborsRegressor(n_neighbors=3).fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
print('The R^2 of the kNN model is: {0:.3f}'.format(knn.score(X_test, y_test)))

The R^2 of the kNN model is: -72.630


In [25]:
df_y_pred_knn = pd.DataFrame(y_pred_knn, index=X_test.index, columns=['PRED_ADJ_CLOSE'])
df_y_pred_knn.head(10)

,PRED_ADJ_CLOSE
DATE,
2019-06-21,117.017776
2019-06-24,126.300306
2019-06-25,115.934428
2019-06-26,125.719752
2019-06-27,110.279134
2019-06-28,125.961502
2019-07-01,124.769997
2019-07-02,125.324760
2019-07-03,126.300306


### Model 3:  Ridge Regression
`https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html`

In [26]:
ridge = Ridge(random_state=42).fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)
print('The R^2 of the ridge model is: {0:.3f}'.format(ridge.score(X_test, y_test)))

The R^2 of the ridge model is: -70.465


In [27]:
df_y_pred_ridge = pd.DataFrame(y_pred_ridge, index=X_test.index, columns=['PRED_ADJ_CLOSE'])
df_y_pred_ridge.head(10)

,PRED_ADJ_CLOSE
DATE,
2019-06-21,117.781720
2019-06-24,122.558465
2019-06-25,121.053035
2019-06-26,116.743944
2019-06-27,113.265293
2019-06-28,119.064801
2019-07-01,118.661232
2019-07-02,122.672532
2019-07-03,122.286511
